In [1]:
import pandas as pd
import numpy as np
# Departments entries - department_id, name
depts = {
'CS': "Department of Computer Science and Engineering",
'EE': "Department of Electronics and Electrical Engineering" ,
'ME': "Department Of Mechanical Engineering" ,
'CE': "Department of Civil Engineering" ,
'DD': "Department of Design" ,
'BT': "Department Of Biosciences and Bioengineering",
'CL': "Department Of Chemical Engineering" ,
'PH': "Department Of Physics" ,
'CH': "Department Of Chemical Sciences and Technology" ,
'MA': "Department Of Mathematics and Computing" ,
'HS': "Department Of Humanities and Social Sciences" ,
'RT' : "Centre for Rural Technology",
'EN' : "Center for Energy",
'LS' : "Centre for Linguistic Science and Technology"
}

with open("150101021_depts.sql","w") as f:
    for d,dname in depts.items():
        f.write("INSERT into Department (department_id,name) values ('"+d+"','"+dname+"');\n")

# Slot entries
final_slots=[]
times=[]
slots_all=pd.read_csv("csvs/slots.csv",header=None)
for i,r in slots_all.iterrows():
    lr= list(r)
    if(i<1):
        times=lr[1:]
    else:
        day = lr[0]
        for t,l in enumerate(lr[1:]):
            if(type(l)==str): #ignore nan
                final_slots.append([l,day]+times[t].split("-"))
len(final_slots)
with open("150101021_slots.sql","w") as f:
    for s in final_slots:
        vals=",".join(map(lambda x:("'"+str(x)+"'"),s));
        f.write("INSERT into Slot (letter,day,start_time,end_time) values (" + vals+");\n")

In [2]:
#Process these 5 files -
filenames= ["Mon","Tue","Wed","Thur","Fri"]
slotof={}
roomof={}
slotdayof={}
rooms_all = set()
for filename in filenames:
    csv = pd.read_csv("csvs/"+filename+".csv")
    
    slots = list(csv.keys())[1:]
    for slot in slots:
        # For use in making entries -
        slotday=filename+"_"+slot
        for i,row in csv[['Room',slot]].dropna().iterrows():
            c = row[1]
            if( c in slotdayof ):
                slotdayof[c].add(slotday);
#                 if(slotday in slotdayof[c]):  
#                     print("Duplicate entry, Must have multirooms",slotday,c)
            else:
                slotdayof[c]=set([slotday])
            
            r = row[0]
            rooms_all.add(r) # distinct rooms
            if( c in roomof ):
                roomof[c].add(r);
            else:
                roomof[c]=set([r])
            # Just for listing the multislots
#             if( c in slotof ):
#                 slotof[c].add(slot);
#             else:
#                 slotof[c]=set([slot])
            
# print("MultiSlot Courses-",[x for x in slotof.items() if(len(x[1]) != 1)])
# print("SingleSlots in variableSlot Courses-",[x for x in slotof.items() if(len(x[1]) == 1)])
print("MultiRoom Courses-",[x for x in roomof.items() if(len(x[1]) != 1)])

MultiRoom Courses- [('EE230', {'4101', '2202', '3101', '2201', '2002'}), ('EE221', {'4101', '2202', '3101', '2201', '2002'}), ('ME222', {'1005', '1003'}), ('CE646', {'4211', '4208', '4205'}), ('CE584', {'4211', '4205'}), ('ME223', {'1005', '1003'}), ('EE663', {'3101', '2002'}), ('EE652', {'4101', '2202'}), ('ME224', {'1005', '1003'}), ('CE307', {'3102', '4201'}), ('CE570', {'4208', '4205'}), ('CE615', {'4211', '4205'}), ('CE623', {'4208', '3102'}), ('PH412', {'4212', '4004'}), ('ME323', {'1005', '1003'}), ('ME324', {'1005', '1003'}), ('ME322', {'1005', '1003'}), ('ME321', {'1005', '1003'}), ('EE551', {'4101', '2202'}), ('ME221', {'1005', '1003'}), ('ME605', {'1G2', '1005'}), ('PH521', {'4212', '4004'}), ('PH531', {'4212', '4004'}), ('CE304', {'4211', '4201'}), ('PH544', {'4212', '4004'}), ('PH545', {'4212', '4004'}), ('ME325', {'1005', '1003'})]


In [3]:
# Course	ClassSlot	Classroom	division	Day   Department
final = []
days={"Mon":"Monday", "Tue":"Tuesday", "Wed":"Wednesday", "Thur":"Thursday", "Fri":"Friday" }
slotbased = pd.read_csv("csvs/Slotbased.csv")

exceptions={
"Tue_E": ['HS203', 'HS212', 'HS213', 'HS220', 'HS224', 'HS226', 'HS234', 'HS237', 'HS238', 'HS239', 'HS242', 'HS243'],
"Mon_A": ['HS406', 'HS409', 'HS413', 'HS414', 'HS415', 'HS420', 'HS421', 'HS422', 'HS424', 'HS425'],
"Tue_E1": ['EE657','CL625','PH446','CH419'],
"Thu_C": ['BT630', 'BT632', 'CE552', 'CE616', 'CH438', 'CL639', 'CS526', 'EE664', 'EE671', 'EN671', 'LS621', 'ME609', 'PH443']
}

# Normal, fixed slot courses-
for i,csrd in slotbased.iterrows():
    course = csrd[0]
    slot = csrd[1]
    rooms_all.add(csrd[2])
    slot_days = [ f[1] for f in final_slots if f[0]==slot ] #only those days which have the slot
    for day in slot_days:
        checkstr = day[:3]+"_"+slot
        if (not (checkstr in exceptions and course in exceptions[checkstr] ) ):
            division = csrd[3]
            if(type(division) != str):
                division = "NA"
            final.append(list(csrd)[:3]+[division,day])
#         else:
#             print("Excluded "+checkstr+" for course "+course)
print("Normal ScheduledIn entries: ",len(final))

# Variable slot courses- (some are fixed though)
for csd in slotdayof.items():
    for s in csd[1]:
        for r in roomof[csd[0]]: #There are multiRoom courses as well.
            sd = s.split("_")
            final.append([csd[0],sd[1],r,'NA',days[sd[0]]])

print("Total ScheduledIn entries: ",len(final))
with open("150101021_sched.sql","w") as f:
    for rel in final:
        c,l,r,dv,dy = rel
        did = c[:2]
        f.write("INSERT INTO ScheduledIn (course_code, course_division, slot_letter, slot_day, department_id, room_number)  VALUES ('"+
                c+"','"+dv+"','"+l+"','"+dy+"','"+did+"','"+r+"');\n")
final[0:1230:200]

Normal ScheduledIn entries:  648
Total ScheduledIn entries:  1282


[['BT101', 'D', 'L2', 'I', 'Monday'],
 ['CH643', 'C', '4005', 'NA', 'Thursday'],
 ['HS220', 'E', '1207', 'NA', 'Friday'],
 ['MA691', 'E1', '1104', 'NA', 'Wednesday'],
 ['PH706', 'D1', '4004', 'NA', 'Thursday'],
 ['PH702', 'A1', '4004', 'NA', 'Wednesday'],
 ['PH544', 'D1', '4212', 'NA', 'Friday']]

In [4]:
# Room entries - room_number, location
print("Total rooms ",len(rooms_all))
locations = ['Core-I', 'Core-II', 'Core-III', 'Core-IV', 'LH', 'Local']
with open("150101021_rooms.sql","w") as f:
    for r in rooms_all:
        l = locations[int(r[0])-1] if r[0] in "1234" else ('LH' if r[0]=='L' else 'Local')
        f.write("INSERT into Room (room_number,location) values ('"+r+"','"+l+"');\n")
        
#Course entries - course_code, division. Comes from slotdayof and slotbased (with division) 
courses = set()
division="NA"
# filenames one -
for c in slotdayof.items():    
    courses.add( (c[0],division) )
print("Weekfiles courses",len(courses))
          
for i,csrd in slotbased.iterrows():
    c = csrd[0]
    d = csrd[3]
    if(type(d) != str):
        d = "NA"
    if( (c,d) in courses):
        print("Duplicate!",c,d)
    else:
        courses.add((c,d))
print("Total courses",len(courses))
with open("150101021_courses.sql","w") as f:
    for c,d in courses:
        f.write("INSERT into Course (course_code,division) values ('"+c+"','"+d+"');\n")


Total rooms  55
Weekfiles courses 164
Total courses 338


In [5]:
#checks for integrity 
#All courses link to a dept -
print("Dept present bugcheck [] ==",[f for f  in final if f[0][:2] not in depts])
# "Other than Tuesday" cases - 
for c in ['CE552','CE616','EE657','EE664','EE671','PH443','PH446']:
    print(c,"bugcheck 3 ==",len([f for f  in final if f[0]==c]))
# Elective rooms - Not mentioned for LS621
print("LS621 bugcheck 3 ==",len([f for f  in final if f[2] == 'Elective_xxxx']))
# CommonRoom xxxx
print(" xxxx bugcheck 6 ==" , len([f for f  in final if f[2] == 'xxxx' or f[2] == 'CommonRoom']))

Dept present bugcheck [] == []
CE552 bugcheck 3 == 3
CE616 bugcheck 3 == 3
EE657 bugcheck 3 == 3
EE664 bugcheck 3 == 3
EE671 bugcheck 3 == 3
PH443 bugcheck 3 == 3
PH446 bugcheck 3 == 3
LS621 bugcheck 3 == 3
 xxxx bugcheck 6 == 6


ScheduledIn entries  1282
